The Boston Housing Dataset is a derived from information collected by the U.S. Census Service concerning housing in the area of Boston MA. The following describes the dataset columns:

1. CRIM - per capita crime rate by town
2. ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
3. INDUS - proportion of non-retail business acres per town.
4. CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
5. NOX - nitric oxides concentration (parts per 10 million)
6. RM - average number of rooms per dwelling
7. AGE - proportion of owner-occupied units built prior to 1940
8. DIS - weighted distances to five Boston employment centres
9. RAD - index of accessibility to radial highways
10. TAX - full-value property-tax rate per ten thousand dollar.
11. PTRATIO - pupil-teacher ratio by town
12. B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
13. LSTAT - % lower status of the population
14. MEDV - Median value of owner-occupied homes in $1000's

We are going to predict the value of MEDV by taking the other 13 attributes as input, this problem is a regression task. We will be using root mean square as a performance measure.

This dataset is taken from Kaggle (https://www.kaggle.com/code/prasadperera/the-boston-housing-dataset/input)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
import seaborn as sns

In [2]:
housing = pd.read_csv("housing.csv")
housing.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB


In [4]:
housing.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


From taking a look at the dataset, we can see that there are 13 numerical attributes and one categorical attribute (CHAS). So, when we will be splitting our dataset into training set and testset, we must take care that the values of CHAS is equally distributed in both of them, it should not be like that training set contains all 0s and test set contains all 1s. Considering this we will be using StratifiedShuffleSplit from sklearn to split dataset.

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(housing, housing['CHAS']):
    stratified_train_set = housing.loc[train_index]
    stratified_test_set = housing.loc[test_index]

In [6]:
corr_matrix = housing.corr()
corr_matrix["MEDV"].sort_values(ascending=False)

MEDV       1.000000
RM         0.695360
ZN         0.360445
B          0.333461
DIS        0.249929
CHAS       0.175260
AGE       -0.376955
RAD       -0.381626
CRIM      -0.388305
NOX       -0.427321
TAX       -0.468536
INDUS     -0.483725
PTRATIO   -0.507787
LSTAT     -0.737663
Name: MEDV, dtype: float64

From the correlation matrix we can see that MEDV has a strong positive correlation with RM (number of rooms per dwelling) and a strong negative correlation with LSTAT. Now, let's see if MEDV has some relation with TAX/RM, that is tax per room.

In [7]:
housing["TAXRM"] = housing["TAX"]/housing["RM"]
corr_matrix = housing.corr()
corr_matrix["MEDV"].sort_values(ascending=False)

MEDV       1.000000
RM         0.695360
ZN         0.360445
B          0.333461
DIS        0.249929
CHAS       0.175260
AGE       -0.376955
RAD       -0.381626
CRIM      -0.388305
NOX       -0.427321
TAX       -0.468536
INDUS     -0.483725
PTRATIO   -0.507787
TAXRM     -0.537650
LSTAT     -0.737663
Name: MEDV, dtype: float64

As we can see that MEDV has a strong negative relation with TAXRM, tax per room. Now, let's do a quick data exploration using ydata_profiling, which will give us the required information about all of the attributes.

In [8]:
profile = ProfileReport(housing)
profile.to_file("housing_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Splitting our training set into inputs and labels

In [9]:
housing = stratified_train_set.drop("MEDV", axis=1)
housing_labels = stratified_train_set["MEDV"].copy()

Since, this dataset does not contain any missing values, we don't have to worry about them, the only thing which we have to take care of while creating pipeline is to scale the attributes. To scale the attributes we will be using StandardScaler from sklearn.

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

housing_tr = pipeline.fit_transform(housing)

Now, our data is scaled, so, we are ready to train models on data, we will be using different models for training, then we will check their performance using performance measure.

## Training models

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

model1 = LinearRegression()
model2 = DecisionTreeRegressor()
model3 = RandomForestRegressor()

model1.fit(housing_tr, housing_labels)
model2.fit(housing_tr, housing_labels)
model3.fit(housing_tr, housing_labels)

RandomForestRegressor()

# Evaluating model

We will be evaluating our models using performance measure and cross validation.

In [12]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_tr = pipeline.fit_transform(some_data)

print(model1.predict(some_data_tr))
print(model2.predict(some_data_tr))
print(model3.predict(some_data_tr))

[20.90447108 28.82280629 15.02261426 24.73759946 23.05953861]
[19.4 33.4 10.5 25.  19.9]
[19.967 31.971  9.555 21.46  21.489]


In [13]:
def evaluate_rmse(model, data, labels):
    from sklearn.metrics import mean_squared_error

    predictions = model.predict(data)
    lin_mse = mean_squared_error(labels, predictions)
    lin_rmse = np.sqrt(lin_mse)
    print("Root mean square is: ",lin_rmse)

In [14]:
def evaluate_cross_val_rmse(model, data, label,n):
    from sklearn.model_selection import cross_val_score
    score = -cross_val_score(model, data, label, scoring="neg_mean_squared_error", cv=n)
    rmse_scores = np.sqrt(score)

    print("Root mean square is: ", rmse_scores)
    print("Mean: ", rmse_scores.mean())
    print("Standard deviation: ", rmse_scores.std())

In [15]:
# Evaluating LinearRegressor using performance measure
evaluate_rmse(model1, housing_tr, housing_labels)
print("\n")

# Evaluating LinearRegressor using cross validation
evaluate_cross_val_rmse(model1, housing_tr, housing_labels, 10)

Root mean square is:  4.829321492635759


Root mean square is:  [4.21674442 4.26026816 5.1071608  3.82881892 5.34093789 4.3785611
 7.47384779 5.48226252 4.14885722 6.0669122 ]
Mean:  5.030437102767305
Standard deviation:  1.0607661158294834


In [16]:
# Evaluating DecisionTreeRegressor using performance measure
evaluate_rmse(model2, housing_tr, housing_labels)
print("\n")

# Evaluating DecisionRegressor using cross validation
evaluate_cross_val_rmse(model2, housing_tr, housing_labels, 10)

Root mean square is:  0.0


Root mean square is:  [4.01497198 5.6742035  5.47426351 3.8621364  4.06813839 3.03973683
 4.60478013 3.75366488 3.38921082 4.24879395]
Mean:  4.2129900384564305
Standard deviation:  0.7956219743765182


In [17]:
# Evaluating RandomForestRegressor using performance measure
evaluate_rmse(model3, housing_tr, housing_labels)
print("\n")

# Evaluating RandomForestRegressor using cross validation
evaluate_cross_val_rmse(model3, housing_tr, housing_labels, 10)

Root mean square is:  1.2111125980880233


Root mean square is:  [2.83454152 2.82557109 4.4327064  2.56632362 3.67603634 2.77113853
 4.84101949 3.26934706 3.25527787 3.21860072]
Mean:  3.3690562640132193
Standard deviation:  0.7085729239188047


LinearRegressor is giving a very high error, so we will have to discard this model. 

DecisonTreeRegressor is giving 0 error, but this means that it is overfitting the data, after using cross validation we can see that this model is performing only slightly better than LinearRegressor.

We can see that RandomForestRegressor is the performing better than both of the previou model, so, now evaluate this model on test set.

# Evaluating our model on test set 

In [18]:
X_test = stratified_test_set.drop("MEDV", axis=1)
y_test = stratified_test_set["MEDV"].copy()
X_test_tr = pipeline.fit_transform(X_test)

In [19]:
test_predictions = model3.predict(X_test_tr)

In [20]:
evaluate_rmse(model3,X_test_tr, y_test)

Root mean square is:  3.4219988153353973


Our model is working fine on test set, now let's save it. 

In [21]:
# saving the model
from joblib import dump, load
dump(model3, 'ml_project_boston_housing.joblib')

['ml_project_boston_housing.joblib']